# Crawler

In [ ]:
# cording: utf-8

## requests

In [ ]:
import requests

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko)     Chrome/48.0.2564.109 Safari/537.36 Vivaldi/1.0.403.24'}
r= requests.get('http://blog.nogizaka46.com/asuka.saito/?d=20160505', headers=headers)

In [ ]:
r.status_code

In [ ]:
contents = r.text
print contents[100:200]

## BeautifulSoup

### document
[Beautiful Soup 4.4.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(contents, 'html.parser')

In [ ]:
tag = soup.find(id='sheet')

In [ ]:
print tag.a.contents[0]

In [ ]:
tag2 = soup.find(id='daytable')
tag2.find_all('a')

In [ ]:
for link in tag2.find_all('a'):
    print link.get('href')

In [ ]:
tag3 = soup.find(id='calendarPN')
prev = tag3.a.get('href')
prev[-4:] == '1111' # 一番初期かどうか
print tag3.a.get('href')

# あすかの

## Database

DBname : Asuka

- Links
  - month : YYmm
  - url : []

In [ ]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import re

In [ ]:
client = MongoClient()
db = client['Asuka']
col = db.Links

In [ ]:
headers ={'User-Agent': 'sample Vivaldi'}
r= requests.get('http://blog.nogizaka46.com/asuka.saito/?d=201112', headers=headers)
r.status_code
text = r.text

In [ ]:
links = ['http://blog.nogizaka46.com/asuka.saito/?d=201605']
crawled = []
# カレンダー判定
def add_link(text):
    soup = BeautifulSoup(text, 'html.parser')
    tag = soup.find(id='calendarPN')
    prev = tag.a.get('href')
    if prev[-4:] == '1111':
        col.insert_one({'month': prev[-4:], 'day_url': []})
        add_day_link(text, prev[-4:])
        crawled.append(prev)
        return 
    else:
        links.append(prev)

def add_day_link(text, month):
    day_add = col.find_one({'month': month})
    soup = BeautifulSoup(text, 'html.parser')
    tag = soup.find(id='daytable')
    for day_link in tag.find_all('a'):
        day_add['day_url'].append(day_link.get('href'))
        col.save(day_add)

while links:
    url = links.pop()
    month = url[-4:]
    crawled.append(url)
    col.insert_one({'month': month, 'day_url': []})
    r = requests.get(url, headers=headers)
    text = r.text
    add_day_link(text, month)
    add_link(text)

In [ ]:
month_1111 = col.find_one({'month': '1112'})
for day_url in month_1111['day_url']:
    print day_url

In [ ]:
def union(p, q):
    for e in q:
        if e not in p:
            p.append(e)

In [ ]:
all_link= []
for a in col.find({'month': re.compile(r'.*')}):
    union(all_link, a['day_url'])
    
all_link

---

In [ ]:
seed = u'http://52.25.113.156'
to_crawl = [seed]
links = soup.find_all('a')
for link in links:
    url = requests.compat.urljoin(seed, link.get('href'))
    if url.find("'")!=-1: continue
    url = url.split('#')[0] # アンカーを取り除く
    if url[0:4] == 'http':
        to_crawl.add(url)
    
to_crawl

In [ ]:
def gettextonly(soup):
    v = soup.string
    if v==None:
        c = soup.contents
        resulttext = ''
        for t in c:
            subtext = gettextonly(t)
            resulttext += subtext + '\n'
        return resulttext
    else:
        return v.strip()

textonly = gettextonly(soup)

## MeCab

In [ ]:
import MeCab

In [ ]:
def split_to_word(text):
    words = []
    m = MeCab.Tagger('-Ochasen')
    text = text.encode('utf-8')
    node = m.parseToNode(text)
    while node:
        words.append(node.surface)
        node = node.next
    return words

text = u"欲望が味方する Wow Wow Wow~"
for i, word in  enumerate(split_to_word(textonly)):
    print i, word

## class Crawler

### description
オライリーの集合知プログラミングの第4章と[Pythonでつくる検索エンジン](http://nwpct1.hatenablog.com/entry/python-search-engine)を参考にしていく。   
コード規約は、[Google Python Style Guide](http://works.surgo.jp/translation/pyguide.html)をみてコーディングを意識したい。

### collection

- url_list
  - url
- word_list
  - word
- word_location
  - url_id
  - word_id
  - location
- link
  - row_id
  - from_id
  - to_id
- link_words
  - word_id
  - link_id

In [ ]:
from pymongo import MongoClient
import requests
from requests.compat import urljoin
from bs4 import BeautifulSoup
import MeCab

In [ ]:
class Crawler:
    # データベースの名前でクローラを初期化する
    def __init__(self, dbname):
        client = MongoClient()
        db = client[dbname]
        self.col = db.Index

    def __del__(self):
        pass
    
    def dbcommit(self):
        pass
    
    # エントリIDを取得したり、それが存在しない場合には追加
    # するための補助関数
    def get_entry_id(self, collection, key, value, createnew=True):
        return None
    
    # 個々のページをインデックスする
    def add_page_to_index(self, url, soup):
        if self.is_indexed(url): return
        print 'Indexing %s' % url

        # 個々の単語を取得する
        text = self.get_text_only(soup)
        words = self.separate_words(text)

        for word in words:
            self.add_to_index(word, url)

    def add_to_index(self, keyword, url):
        entry = self.col.find_one({'keyword': keyword})
        if entry:
            if url not in entry['url']:
                entry['url'].append(url)
                self.col.save(entry)
                return
        # 単語が見つからないから新規作成
        self.col.insert_one({'keyword': keyword, 'url': [url]})
        self.col.create_index('keyword')

    # HTMLのページからタグのない状態でテキストを摘出する
    def get_text_only(self, soup):
        values = soup.string
        if values == None:
            contents = soup.contents
            result_text = ''
            for t in contents:
                sub_text = self.get_text_only(t)
                result_text += sub_text + '\n'
            return result_text
        else:
            return values.strip()
    
    # 空白以外の文字で単語を分割する
    def separate_words(self, text):
        words = []
        m = MeCab.Tagger('-Ochasen')
        text = text.encode('utf-8')
        node = m.parseToNode(text)
        while node:
            words.append(node.surface)
            node = node.next
        return words

    # URLが既にインデックスされていたらtrueを返す
    def is_indexed(self, url):
        return False
    
    # ２つのページ間にリンクを付け加える
    def add_link_ref(self, url_from, url_to, link_text):
        pass
    
    def union(self, p, q):
        for e in q:
            if e not in p:
                p.append(e)
    
    # ページを受け取り、与えられた深さで幅優先の検索を行い
    # ページをインデクシングする
    def crawl(self, seed, max_depth=2):
        pages = [seed]
        crawled = []
        next_depth = []
        depth = 0
        while pages and depth <= max_depth:
            page_url = pages.pop()
            if page_url not in crawled:
                res = requests.get(page_url)
                try:
                    res.status_code == 200
                except:
                    print 'Could not open %s' % page
                    continue
                html_doc = res.text
                soup = BeautifulSoup(html_doc, 'html.parser')
                self.add_page_to_index(page_url, soup)
                links = []
                for link in soup.find_all('a'):
                    link_url = urljoin(page_url, link.get('href'))
                    if link_url.find("'")!=-1: continue
                    if link_url[-1] == '/':
                        link_url = link_url[:-1] # 最後が'/'で終わる場合取り除く
                    link_url = link_url.split('#')[0] # アンカーを取り除く
                    link_url = link_url.split('?')[0] # パラメータを取り除く
                    if link_url[0:4] == 'http':
                        links.append(link_url)
                    link_text = self.get_text_only(link)
                    self.add_link_ref(page_url, link_url, link_text)
                
                self.union(next_depth, links)

                crawled.append(page_url)
                print pages
                print next_depth

            if not pages:
                pages, next_depth = next_depth, []
                depth += 1
                print depth

In [ ]:
crawler = Crawler('Crawler')

In [ ]:
crawler.crawl('http://52.25.113.156', 2)

In [ ]:
s = 'test/'
if s[-1] == '/':
    s = s[:-1]

s

In [ ]:
crawler.db['word_list']

In [ ]:
def union(p,q):
    for e in q:
        if e not in p:
            p.append(e)

In [ ]:
pages = ['http://52.25.113.156']
type(pages)
union(pages, [u'http://52.25.113.156/users/sign_up', u'http://52.25.113.156/users/sign_in', u'http://52.25.113.156/tops/about', u'http://52.25.113.156', u'http://52.25.113.156/posts/4', u'http://52.25.113.156/posts/5', u'http://52.25.113.156/users/auth/twitter', u'http://52.25.113.156/posts/3', u'http://52.25.113.156/posts/1'])
pages